# Bart WalkThrough
> A journey through the forward pass

- toc: true 
- badges: true
- comments: true
- categories: [jupyter]
- image: images/chart-preview.png

### Seq2Seq Pretraining
In October 2019, teams from Microsoft, Google and Facebook independently published three new transformer papers: UniLM, T5 and Bart. All three papers analyze found, in general that they achieve better downstream performance if they (a) replace Bert's bidirectional architecture with a seq2seq architecture and (b) Bert's fill-in-the blank cloze task with a more complicated mix of pretraining tasks. It's a fun game to try to match which of the following quotes from the abstract map to which paper:


1. 
> While many modern approaches to transfer learning for NLP use a Transformer architecture consisting of only a single “stack” (e.g. for language modeling [GPT2]  or classification and span prediction [BERT]), we found that using a standard encoder-decoder structure achieved good results on both generative and classification tasks. 

2. 
> The model is pre-trained using three types of language modeling tasks: unidirectional, bidirectional, and sequence-to-sequence prediction.

3. 
> We present a denoising autoencoder for pretraining sequence to sequence models, ... it uses a standard Transformer-based neural machine translation architecture.

Answers: [^1].

 [^1]: *Answers: (T5, Oct 24) , (UniLM, Oct. 15) , (Bart, Oct. 29)

If you got that right you are very lucky indeed!

So lets dig a bit deeper on this big Seq2Seq idea, then dive into some interesting parts of the code!

### Big Idea: Bidirectional Encoder, Left-To-Right Decoder
Bert is pretrained to try to predict mask tokens, and uses the whole sequence to get enough info to make a good guess. This is good for tasks where the prediction at position `i` can depend on information from positions after `i`, but suboptimal for tasks where you are not, like text generation, where you generate the next word conditional on the words you have seen BEFORE.

In the code, bert's "Fully Visible" `attention_mask` is boring:

In [50]:
%%html 

#hide-input
<img src="bert_attn_mask.jpg" alt="Drawing" style="width: 200px;"/>

 GPT2, meanwhile, is pretrained to predict the next word.
This makes it good at generation tasks, where there aren't future tokens to consider, but suboptimal for other downstream tasks where the causal mask provides no benefit.

Here is the attention mask for GPT2, white squares in y2, x3 show that the prediction for timestep 2 does not depend on the input at timestep 3.

In [54]:
%%html 

#hide-input
<img src="gpt2_simple_mask.jpg" alt="Drawing" style="width: 200px;"/>

In [52]:
%%html 

#hide-input
<img src="gpt2_simple_mask.jpg" alt="Drawing" style="width: 200px;"/>

Our new friends get the best of both worlds:

The encoder is bidirectional  - each token's attention can attend to every other token in the input sequence, while the decoder, which will ultimately have to perform generation, is causal like GPT2.

We can think about this attention pattern as smushing together our previous two attention masks, or "Causal Mask  with a fully visible prefix" in fancier terms.[^4]

[^4]: The indices dont line up perfectly for the smush to work, but tokens 1 and 2 are the fully visible prefix (or the input to the encoder) and tokens 3,4,5 are the causally masked suffix (or inputs to the decoder). In summarization terms, you could imagine tokens 1 and 2 as the article, and we generate tokens 3-5 auto-regressively.


![](./t5_mask_diagram.png)


This attention pattern is very well suited to summarization and other conditional generation tasks.  You are allowed to attend to the whole document, but as you write your summary, one word at a time, you need only consider what you've already written. The numbers confirm this: all the new fancy guys do a lot better than the old less-fancy guys.

In [36]:
#collapse-hide 
import pandas as pd
pd.read_csv('tab1.csv', index_col=0)

,CNNDM Rouge 2 score
Paper,
Bart,21.28
UniLM,20.30
BertSumABS,19.39
t5-base,20.34
t5 11B,21.55
TransformerAbs (2018),17.76


<!-- I start with this not to say that the researchers are lame, but that they were all discovering a very cool idea at the same time. In the following table, you can see that, at least for summarization, it is important to pretrain with a bidirectional encoder and causal decoder: (The UniLM `prefix_lm` is equivalent [^3]) -->

`BertSumABS` [^2] , exploits the Seq2Seq architecture but doesn't pretrain the decoder.
Also note that t5-11b is 22x bigger than Bart), and pretraining objectives.

Bart tries out crazy pretraining tasks that you can only do with a seq2seq architecture. Since "Inputs to the encoder need not be aligned with decoder outputs, allowing arbitary noise transformations." They invent a pretraining task called Text Infilling, where you
replace a **span** of text with a single mask token. This span can be of any length, so the model also must learn how many tokens to generate.

There is also another trick in Bart: each decoder layer performs cross-attention over the final hidden state of the encoder output. This presumably nudges Bart towards generating summaries that are closely connected to the original (encoded) text.

#### Awkward Transition to Eng

Shortly after these papers were released our `transformers` users started asking for us to make them available in the repo, especially Bart. And now, a few months later, it's demo time!



### `transformers.BartForConditionalGeneration`

#### imports

In [77]:
#collapse-hide
import torch
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
LONG_ARTICLE = ' (CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC\'s founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, including East Jerusalem, since June 13, 2014." Later that month, the ICC opened a preliminary examination into the situation in Palestinian territories, paving the way for possible war crimes investigations against Israelis. As members of the court, Palestinians may be subject to counter-charges as well. Israel and the United States, neither of which is an ICC member, opposed the Palestinians\' efforts to join the body. But Palestinian Foreign Minister Riad al-Malki, speaking at Wednesday\'s ceremony, said it was a move toward greater justice. "As Palestine formally becomes a State Party to the Rome Statute today, the world is also a step closer to ending a long era of impunity and injustice," he said, according to an ICC news release. "Indeed, today brings us closer to our shared goals of justice and peace." Judge Kuniko Ozaki, a vice president of the ICC, said acceding to the treaty was just the first step for the Palestinians. "As the Rome Statute today enters into force for the State of Palestine, Palestine acquires all the rights as well as responsibilities that come with being a State Party to the Statute. These are substantive commitments, which cannot be taken lightly," she said. Rights group Human Rights Watch welcomed the development. "Governments seeking to penalize Palestine for joining the ICC should immediately end their pressure, and countries that support universal acceptance of the court\'s treaty should speak out to welcome its membership," said Balkees Jarrah, international justice counsel for the group. "What\'s objectionable is the attempts to undermine international justice, not Palestine\'s decision to join a treaty to which over 100 countries around the world are members." In January, when the preliminary ICC examination was opened, Israeli Prime Minister Benjamin Netanyahu described it as an outrage, saying the court was overstepping its boundaries. The United States also said it "strongly" disagreed with the court\'s decision. "As we have said repeatedly, we do not believe that Palestine is a state and therefore we do not believe that it is eligible to join the ICC," the State Department said in a statement. It urged the warring sides to resolve their differences through direct negotiations. "We will continue to oppose actions against Israel at the ICC as counterproductive to the cause of peace," it said. But the ICC begs to differ with the definition of a state for its purposes and refers to the territories as "Palestine." While a preliminary examination is not a formal investigation, it allows the court to review evidence and determine whether to investigate suspects on both sides. Prosecutor Fatou Bensouda said her office would "conduct its analysis in full independence and impartiality." The war between Israel and Hamas militants in Gaza last summer left more than 2,000 people dead. The inquiry will include alleged war crimes committed since June. The International Criminal Court was set up in 2002 to prosecute genocide, crimes against humanity and war crimes. CNN\'s Vasco Cotovio, Kareem Khadder and Faith Karimi contributed to this report.' 
LONG_ARTICLE

' (CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC\'s founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, including East Jerusalem, since June 13, 2014." Later that month, the ICC opened a preliminary examination into the situation in Palestinian territories, paving the way for possible war crimes investigations against Israelis. As members of the court, Palestinians may be subject to counter-charges as well. Israel and the United States, neither of which is an ICC member, opposed the Palestinians\' efforts to join the body. But Palestinian Foreign Minister Riad al-Malki, speaking at Wednesday\'s ceremon

In [63]:
tokenizer = BartTokenizer.from_pretrained('bart-large-cnn')
article_input_ids = tokenizer.batch_encode_plus([LONG_ARTICLE], return_tensors='pt')['input_ids'].to(torch_device)
model = BartForConditionalGeneration.from_pretrained('bart-large-cnn')
summary_ids = model.generate(article_input_ids, num_beams=4, length_penalty=2.0,
                             max_length=140, min_len=55)
tokenizer.decode(summary_ids.squeeze(), )

"<s>The Palestinian Authority becomes the 123rd member of the International Criminal Court. The move gives the court jurisdiction over alleged crimes in Palestinian territories. Israel and the United States opposed the Palestinians' efforts to join the body. But Palestinian Foreign Minister Riad al-Malki said it was a move toward greater justice."

**TODO**: get conditional generation working on GPT2 for this Doc. The following snippet goes straight to EOS.
```python
from transformers import GPT2LMHeadModel, GPT2Tokenizer
gpt2_tok = GPT2Tokenizer.from_pretrained('gpt2')
gpt2_model = GPT2LMHeadModel.from_pretrained('gpt2')

article_input_ids = gpt2_tok.batch_encode_plus([LONG_ARTICLE], return_tensors='pt', pad_to_max_length=False)['input_ids'].to(torch_device)
summary_ids = gpt2_model.generate(article_input_ids, max_length=article_input_ids.shape[1] + 155, do_sample=False)
gpt2_tok.decode(summary_ids.squeeze(), ).split('\n')
```

One thing to notice in these two snippets: even though `BartForConditionalGeneration` is a seq2seq model, and `GPT2LMHeadModel` is not, they can be used in very similar ways for generation.

The same correspondence exists between `BartForSequenceClassification`  and `BertForSequenceClassification`.  

It makes sense that you would want an Encoder and a Decoder if you wanted the number of stuffs you generate not to depend on the number of stuffs you take as input. You could even understand, in that world, having an API that was like
```python
def forward(self, 
            encoder_input_ids, 
            encoder_padding_mask, 
            decoder_input_ids, 
            decoder_causal_attention_mask,
            decoder_padding_mask):
```
But if you live in a world, like many of our users, where you expect your model to tell you the sentiment of a movie review, or the most likely next word, you would be like WTF is a decoder and why do I care: I just want to pass in a movie review my long winded friend Tim texted me and get a number so that I can decide whether to watch *The Notebook*.

I want a signature like
```python
def forward(
    self,
    input_ids,
    encoder_padding_mask=None,
    decoder_input_ids=None,
    decoder_padding_mask=None, ):
```
The way Bart makes decoder_input_ids, in this case, is by taking the input_ids (movie review) and shifting them to the right. The authors' [motivation](https://github.com/pytorch/fairseq/issues/1389) for that trick was to facilitate teacher forcing during pre-training, and now that the model has been trained on 64 TPUs for 12 weeks to process this input format, we continue the pattern during inference, but hide it inside the forward method. We also hide the creation of the other decoder padding masks so that at least we are consistent within one signature (rewrite at least last sentence) [^1]
Now let's move back to our seq2seq world, so that we can generate a shorter version of Tim's movie review and make him think we read his review, watched The Notebook, and have a lot in common.

> Note 
Most of our other models do not make inputs for the user -- that's the tokenizer's job, but as the t5 authors write:

> "A major factor that differentiates the architectures is the mask used by different attention mechanisms in the model." 


### Trick 2: `super_fast_cached_generation_mode`

When I first read the fairseq code, there was a function called `make_generation_fast` which didnt do much besides catch my eye. What an exciting name! 
Anyways, here is a really slow (pseudocode) way to generate summaries
    
```python
summary_so_far = []
while not done:
     encoder_hidden_state = model.encoder(article_input_ids)
     logits = model.decoder(encoder_hidden_state, summary_so_far)
     next_word = logits.argmax()
     summary_so_far.append(next_word)
```

We can just cache the first step and save half the compute

```python
summary_so_far = []
encoder_hidden_state = model.encoder(article_input_ids)
while not done:
     logits = model.decoder(encoder_hidden_state, summary_so_far)
     next_word = logits.argmax()
     summary_so_far.append(next_word)
```

but we're not done yet! Remember, inside each layer of `model.decoder` are two types of attention: while the `encoder_attention` modules look at `encoder_hidden_state`, while the `self_attention` modules look at `summary_so_far`. For the first type of attention, we can basically cache everything; it's not gunna change during our loop. For self attention, we can cache the keys and values associated with tokens we've already generated, so for each forward pass, all we need to do is compute `q`, `k`, and `v` [^2] 
```python
            k = torch.cat((past_key, k), dim=-2) # the seq_len dimension
            v = torch.cat((past_value, v), dim=-2)
```
[^2] lol

**V1: cache encoder_hidden_state**
```
generated = []
encoder_hidden_state = model.encoder(article_input_ids)
while not Done:
     logits = model.decoder(encoder_hidden_state, generated)
     
     next_word = logits.argmax()
     generated.append(next_word)
     
```


**V2: cache attention (i,j) for j < t**
```
generated = []
encoder_hidden_state = model.encoder(article_input_ids)
while not Done:
     logits = model.decoder(encoder_hidden_state, generated)
     
     next_word = logits.argmax()
     generated.append(next_word)
     
```


In [25]:
from transformers import *

In [3]:
ls

2020-02-20-test.ipynb  README.md              my_icons/
2020-03-06-bart.ipynb  gpt2_mask.jpeg


In [24]:
article_input_ids = tokenizer.batch_encode_plus([SHORTER_ARTICLE], return_tensors='pt')['input_ids']
summary_ids = model.generate(article_input_ids, length_penalty=2.0,
                             max_length=140, min_len=55)

In [23]:
summary_ids = model.generate(article_input_ids, num_beams=4, length_penalty=2.0,
                             max_length=140, min_len=55)
tokenizer.decode(summary_ids.squeeze(), )

"<s>The Palestinian Authority becomes the 123rd member of the International Criminal Court. The move gives the court jurisdiction over alleged crimes in Palestinian territories. Israel and the United States opposed the Palestinians' efforts to join the body. But Palestinian Foreign Minister Riad al-Malki said it was a move toward greater justice."

## Footnotes

[^2] "Text Summarization with Pretrained Encoders" https://arxiv.org/abs/1908.08345

[^3] Differences between the UniLM Masking strategy and Bart

### Appendix
 ![](./bert_attn_mask.jpg)

![](./gpt2_mask.jpeg) "Left-to-right mask. For a given token in the sequence, we assign a mask value of 0 for this token and the preceding ones; a value of minus infinity for the later ones. This pretty square can also be called and `causal_mask` autoregressive mask. (Picture Drawn by Remy Louf)"

## Boxes / Callouts 

Typing `> Warning: There will be no second warning!` will render this:


> Warning: There will be no second warning!



Typing `> Important: Pay attention! It's important.` will render this:

> Important: Pay attention! It's important.



Typing `> Tip: This is my tip.` will render this:

> Tip: This is my tip.



Typing `> Note: Take note of this.` will render this:

> Note: Take note of this.



Typing `> Note: A doc link to [an example website: fast.ai](https://www.fast.ai/) should also work fine.` will render in the docs:

> Note: A doc link to [an example website: fast.ai](https://www.fast.ai/) should also work fine.

## Footnotes

You can have footnotes in notebooks just like you can with markdown. 

For example, here is a footnote [^1].

[^1]: This is the footnote.